# Aula de laboratório - Estimação Pontual e Intervalar

Rafael Bressan
24/04/2023

# Introdução

Nesta aula vamos utilizar dados disponíveis do livro _Data Analysis for 
Business, Economics, and Policy_ (Békés e Kézdi, 2021). Os estudos de caso e
dados disponibilizados pelos autores encontram-se no site [https://gabors-data-analysis.com/](https://gabors-data-analysis.com/).

Como pano de fundo para nossa aula de laboratório utilizaremos a estudo de 
caso _"Comparing hotel prices in Europe: Vienna vs London"_.

# Estudo de Caso

Como podemos comparar os mercados hoteleiros na Europa e conhecer as características 
dos preços dos hotéis? Podemos visualizar duas distribuições em um gráfico? 
Que estatísticas descritivas melhor descreveriam cada distribuição e suas 
diferenças? Podemos visualizar estatísticas descritivas?

Este estudo de caso usa o conjunto de dados da `hotels-europe` e seleciona 
hotéis de 3 a 4 estrelas em Viena e Londres para comparar a distribuição de 
preços para um dia da semana em novembro de 2017. Ele ilustra a comparação de 
distribuições e o uso de histogramas e gráficos de densidade. Ele ilustra o 
uso de algumas das estatísticas descritivas mais importantes para variáveis 
quantitativas e suas visualizações, box plots e violin plots. 

Além das comparações gráficas, utilizaremos testes formais de hipóteses para
comparar o preço médio de hospedagem em Viena e Londres.

## Apresentando o conjunto de dados hotels-europe

Os dados da `hotels-europe` incluem informação sobre preços e características 
de hotéis em 46 cidades europeias e para 10 datas diferentes. N=148.021. 
Variáveis chave: preço do hotel, distância do hotel ao centro da cidade. 
Extraído de um site de comparação de preços. Foi anonimizado e ligeiramente 
alterado para garantir a confidencialidade. Ele contém muitas informações 
sobre o local e a classificação, mas não o nome ou o endereço.

# Código Python

Novamente, começamos carregando as bibliotecas necessárias para nossa análise.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotnine import *
from scipy.stats import skew, kurtosis, ttest_1samp, ttest_ind

Vamos baixar os dados e salvá-los em um Data Frame. Este conjunto de dados é
na verdade **composto por duas tabelas** que possuem uma coluna em comum, 
também conhecida como chave primária na terminologia de banco de dados. É a partir
desta coluna (`hotel_id`) que iremos unir os dois data frames em um único. O 
comando `head()` nos fornecerá uma visualização rápida dos dados.


In [ ]:
precos = pd.read_csv("https://osf.io/p6tyr/download/")
precos.head()

In [ ]:
hoteis = pd.read_csv("https://osf.io/utwjs/download/")
hoteis.head()

Precisamos fazer a união destes dados (merge ou join). Antes vamos filtar as datas e as cidades de interesse, para somente depois fazer a união dos dados e obter apenas um data frame para trabalhar.


In [ ]:
precos_filtrado = precos.loc[(precos["year"] == 2017) & (precos["month"] == 11), :]

filtro_cidade = hoteis["city"].isin(["Vienna", "London"])
filtro_estrelas = (hoteis["stars"] >= 3) & (hoteis["stars"] <= 4)
hoteis_filtrado = hoteis.loc[filtro_cidade & filtro_estrelas, :]

# Inner Join: apenas os hoteis que estao em ambos data frames
df = pd.merge(precos_filtrado, hoteis_filtrado, on="hotel_id")

In [ ]:
# Apenas uma conferencia para ver se as filtragens foram corretas
[df[x].unique() for x in ["city", "year", "month", "stars"]]

Temos agora um data frame completo (`df`) com todos os dados necessários para nossa análise. Começaremos com alguns típicos gráficos para analisar distribuição de variáveis quantitativas.

## Histogramas

In [ ]:
histpreco = (ggplot(df, aes(x ='price', y = after_stat('density'))) +
  geom_histogram(binwidth = 20, 
                 fill = "darkred", color = "white") +
  labs(x = "Preco (US dolares)", y = "Densidade") +
  scale_x_continuous(limits = (0, 500),
                     breaks = np.arange(0, 500, step=50)) +
  facet_wrap('~city', ncol = 1) +
  theme_classic()
)
histpreco


Suponha agora que nestes histogramas desejamos acrescentar uma linha vertical indicando o preço médio das estadias. Esta é uma forma de ajudar na visualização da diferença de um importante parâmetro. Precisamos primeiro calcular estas médias por cidade e guardar em um data frame separado


In [ ]:
precos_medios = df.groupby('city', as_index=False)[['price']].mean()
precos_medios


E agora utilizamos este novo data frame para incluir as linhas verticais.

In [ ]:
histpreco_media = (histpreco +
  geom_vline(aes(xintercept = 'price'), data = precos_medios,
             linetype = 'dashed')
)
histpreco_media


## Boxplot
 
Outra forma de investigar graficamente as diferenças nas distribuições de preços é através de boxplots, um para cada cidade.


In [ ]:
boxpreco = (ggplot(df, aes(x = 'city', y = 'price')) +
  geom_boxplot(aes(color = 'city'), show_legend = False) +
  labs(y = "Preco (US dolares)", x = "Cidade") +
  theme_classic()
) 
boxpreco


## Violino

Podemos aliar as informações de um boxplot ao formato de um gráfico de densidade e obter o chamado gráfico de violino.


In [ ]:
violinopreco = (ggplot(df, aes(x = 'city', y = 'price')) +
  geom_violin(aes(color = 'city'), show_legend = False,
              draw_quantiles = (0.25, 0.5, 0.75)) +
  labs(y = "Preco (US dolares)", x = "Cidade") +
  coord_flip() +
  theme_classic() 
)
violinopreco


## Estatísticas Descritivas

Percebemos claramente através de todos os gráficos apresentados que as distribuições de preços são diferentes entre as cidades. Valores mínimos, máximos,média e de observações podem ser calculados para cada uma das cidades e tabulados.
Além destes, vamos computar também alguns outros momentos da distribuição:variância, assimetria e curtose.


In [ ]:
descritivas = (df.groupby('city', as_index=False)["price"]
               .agg(['mean', 'min', 'max', 'count', 'var', skew, kurtosis])
               .rename(columns={"mean": "Media"})
)

descritivas


## Teste de Hipótese Paramétrico

Por ora vamos testar os preços médios de estadia nas cidades de forma individual contra valores fixos. Iremos fazer um **teste bilateral** em Viena, verificando se o preço médio é diferente de US\$ 150 enquanto que para Londres faremos um teste unilateral, $H_0: \mu < 200$.

Temos uma amostra de dados onde podemos estimar a média, recaímos no Caso I da aula. Este é o famoso caso do **teste t** para apenas uma amostra. 

No `Python` podemos fazer o teste t através da função `ttest_1samp()` da biblioteca `scipy`.


In [ ]:
precos_viena = df.loc[df['city']=='Vienna', 'price']
viena_ttest = ttest_1samp(precos_viena, popmean=150)
viena_ttest

In [ ]:
precos_londres = df.loc[df['city']=='London', 'price']
ttest_1samp(precos_londres, popmean=200, alternative='less')

O intervalo de confiança para um teste t feito com esta função pode ser obtido através do método `confidence_interval`.

In [ ]:
viena_ttest.confidence_interval(0.9)

Podemos fazer agora um teste de **diferença de médias**. O valor médio da hospedagem em Londres é estatisticamente mais caro que em Viena? Já sabemos que em média Londres é mais caro, mas podemos nos perguntar se esta diferença é estatisticamente significativa. Para tanto, faremos um teste unilateral sobre a diferença entre Londres e Viena, com a hipótese alternativa sendo "a diferença é maior que zero".

In [ ]:
diferenca_ttest = ttest_ind(precos_londres, precos_viena, equal_var=False, alternative='greater')
diferenca_ttest